In [7]:
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

In [3]:
!pip install transformers accelerate peft datasets bitsandbytes torch

In [4]:
import pandas as pd
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login

from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
MODEL_NAME = "deepseek-ai/deepseek-math-7b-base"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
model = prepare_model_for_kbit_training(model)

In [7]:
import re
def get_num_layers(model):
    numbers = set()
    for name, _ in model.named_parameters():
        for number in re.findall(r'\d+', name):
            numbers.add(int(number))
    return max(numbers)

def get_last_layer_linears(model):
    names = []
    
    num_layers = get_num_layers(model)
    for name, module in model.named_modules():
        if str(num_layers) in name and not "encoder" in name:
            if isinstance(module, torch.nn.Linear):
                names.append(name)
    return names

In [8]:
config = LoraConfig(
    r=20,
    lora_alpha=40,
    target_modules=get_last_layer_linears(model),
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

# Data

In [9]:
df = pd.read_csv('dart_math_bangla.csv')
df.head()

,Unnamed: 0,query,eng_query,response,eng_response
0,167855,$f: \mathbb{R} \to \mathbb{R}$ এমন একটি ফাংশন ...,Let $f : \mathbb{R} \to \mathbb{R}$ be a funct...,"$x = y = 0,$ সেট করলে আমরা \[f(0) ^2 - f(0) = ...","Setting $x = y = 0,$ we get\n\[f(0)^2 - f(0) =..."
1,199546,কোরি ১ থেকে ২৫ এর মধ্যে সমস্ত মৌলিক সংখ্যার এক...,Cory made a complete list of the prime numbers...,1 থেকে 25 এর মধ্যে সবচেয়ে ছোট মৌলিক সংখ্যা হল...,The smallest prime number between 1 and 25 is ...
2,28560,${}-27) ^{5/3}$ এর মান নির্ণয় করুন।,Evaluate $(-27)^{5/3}$.,আমরা $(-27) ^{5/3} = ((-3) ^3) ^{5/3}$. সম্পত্...,We have $(-27)^{5/3} = ((-3)^3)^{5/3}$.\nUsing...
3,8415,${1}{2}{3}{2}{4}{3}{4}{4}}{5}{4}}$ সরলীকরণ করুন।,Simplify $(1)(2a)(3a^2)(4a^3)(5a^4)$.,কোয়েফিশেন্টের গুণঃ $1\cdot2\cdot3\cdot4\cdot5...,Multiplying the coefficients: $1\cdot2\cdot3\c...
4,492063,র্যান্ডির কাছে ৩০০০ ডলার ছিল। স্মিথ তাকে আরও ২...,"Randy had $3,000. Smith gave him another $200....","র্যান্ডি শুরু থেকে ৩০০০ ডলার দিয়ে শুরু করে, স...","Randy started with $3,000 from the beginning, ..."


In [11]:
data = Dataset.from_pandas(df)

# Sample Generation

In [12]:
prompt = "Problem: " + df["query"].values[0] + " ".strip()
prompt

'Problem: $f: \\mathbb{R} \\to \\mathbb{R}$ এমন একটি ফাংশন হতে হবে যে \\[f(x) f(y) - f(xy) = x + y\\] সব বাস্তব সংখ্যার জন্য $x$ এবং $y.$ $n$ হতে হবে $f(2),$ এর সম্ভাব্য মানের সংখ্যা, এবং $s$ হতে হবে $f(2) এর সম্ভাব্য মানের যোগফল।$ Find $ntimes s.$'

In [21]:
df['response'].values[0]

'$x = y = 0,$ সেট করলে আমরা \\[f(0) ^2 - f(0) = 0,\\] সুতরাং $f(0) (f(0) - 1) = 0.$ সুতরাং, $f(0) = 0$ অথবা $f(0) = 1.$ সেট করলে $y = 0,$ আমরা \\[f(x) f(0) - f(0) = x,\\] সুতরাং $f(x) = x + 1.$ এই ফাংশনটি কাজ করে, তাই $n = 1$ এবং $s = 3,$ এবং $n \\times s = 3.$ উত্তর হল: 3.'

In [13]:
generation_config = model.generation_config
generation_config.max_new_tokens = 512
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.do_sample = True
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [14]:
%%time
device = "cuda"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model.generate(
        input_ids = encoding.input_ids,
        attention_mask = encoding.attention_mask,
        generation_config = generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Problem: $f: \mathbb{R} \to \mathbb{R}$ এমন একটি ফাংশন হতে হবে যে \[f(x) f(y) - f(xy) = x + y\] সব বাস্তব সংখ্যার জন্য $x$ এবং $y.$ $n$ হতে হবে $f(2),$ এর সম্ভাব্য মানের সংখ্যা, এবং $s$ হতে হবে $f(2) এর সম্ভাব্য মানের যোগফল।$ Find $ntimes s.$
Problem: $f: \mathbb{R} \to \mathbb{R}$ এমন একটি ফাংশন হতে হবে যে \[f(x) f(y) - f(xy) = x + y\] সব বাস্তব সংখ্যার জন্য $x$ এবং $y.$ $n$ হতে হবে $f(2),$ এর সম্ভাব্য মানের সংখ্যা, এবং $s$ হতে হবে $f(2) এর সম্ভাব্য মানের যোগফল।$

CPU times: user 16.4 s, sys: 335 ms, total: 16.7 s
Wall time: 16.7 s


# Encoding

In [15]:
def generate_prompt(data_point):
    return f"""Problem Statement: {data_point["query"]}  
            Solution: {data_point["response"]} """.strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/9064 [00:00<?, ? examples/s]

# Training

In [16]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [17]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=True,
    output_dir="checkpoints",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.01,
    logging_steps=5,
    report_to=None
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
CODECARBON : No CPU tracking mode found. Falling back on CPU constant mode.
huggingface/tokenizers: The current process just got forked, after parallelis

Step,Training Loss
5,0.567600
10,0.529500
15,0.544500
20,0.530800
25,0.491000
30,0.450400
35,0.473200
40,0.469800
45,0.461800
50,0.433800


/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=

TrainOutput(global_step=2266, training_loss=0.340965122747295, metrics={'train_runtime': 10393.3404, 'train_samples_per_second': 0.872, 'train_steps_per_second': 0.218, 'total_flos': 2.122863007029719e+17, 'train_loss': 0.340965122747295, 'epoch': 1.0})

In [18]:
model.save_pretrained("trained-model")

/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
%%time

device = "cuda"
encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Problem: $f: \mathbb{R} \to \mathbb{R}$ এমন একটি ফাংশন হতে হবে যে \[f(x) f(y) - f(xy) = x + y\] সব বাস্তব সংখ্যার জন্য $x$ এবং $y.$ $n$ হতে হবে $f(2),$ এর সম্ভাব্য মানের সংখ্যা, এবং $s$ হতে হবে $f(2) এর সম্ভাব্য মানের যোগফল।$ Find $ntimes s.$  
            Solution: আমরা একটি সহজ উত্তর পাই $f(x) = 0,$ এবং $f(x) = 1,$ উভয় ফাংশনই সমীকরণটি সন্তুষ্ট করে। সুতরাং $n = 2$। এখন আমরা দেখি যে $f(2) = 1.$ যদি $f(2) = 0,$ তাহলে আমরা পাই $0 = 2,$ যা অসম্ভব। সুতরাং $s = 1,$ এবং $ns = 2.$ উত্তর হলঃ $2.$ উত্তর হলঃ $2.$ উত্তর হলঃ $2.$ উত্তর হলঃ $2.$ উত্তর হলঃ $2.$ উত্তর হলঃ $2.$ উত্তর হলঃ $2.$ উত্তর হলঃ $2.$ উত্তর হলঃ $2.$ উত্তর হলঃ $2.$ উ
CPU times: user 4min 53s, sys: 2min 38s, total: 7min 31s
Wall time: 7min 31s
